# Visualize post data from Stack Overflow

## 1. Overview

- The csv file contains records of posts from a Q&A platform (similar to Stack Overflow). Each record corresponds to a single post, which can be either a question or an answer. The dataset includes metadata about the post (ID, type, creation date), user information, content, and community interaction metrics.

## 2. Data Structure and Key Fields

### 2.1. Identification and Classification

- **Id**: Unique identifier of the post.
- **PostTypeId**: Defines the type of the post:
  - **"1"** → Question
  - **"2"** → Answer
- **ParentId**: Only present for answers, referencing the ID of the parent question.

### 2.2. Content Information

- **Title**: Title of the post (only for questions).
- **Body**: Main content of the post, typically in HTML format, which may include paragraphs, code snippets, and formatted text.
- **Tags**: List of topic tags, separated by |, describing the subject areas of the post.

### 2.3. Community Interaction Metrics

- **Score**: Net score of the post (upvotes – downvotes).
- **ViewCount**: Number of times the question was viewed (only for questions).
- **AnswerCount**: Number of answers associated with a question.
- **CommentCount**: Number of comments made on the post.

### 2.4. User and System Metadata

- **CreationDate**: Timestamp (ISO 8601 format) indicating when the post was created.
- **OwnerUserId**: User ID of the author of the post.

## 3. Load data

In [67]:
import pandas as pd

- Function to load data from a CSV file into a DataFrame
    - **Parameters**:
        - **_file_path (str)_**: Path to the CSV file to be loaded
    - **Returns**:
        - **_pd.DataFrame_**: DataFrame containing the data from the CSV file

In [68]:
def load_csv_to_dataframe(file_path):
    try:
        df = pd.read_csv(file_path)
        print("Data has been successfully loaded!")
        print(f"Number row: {df.shape[0]}, Number column: {df.shape[1]}")
        return df
    except FileNotFoundError:
        print("File not found. Please check the file path!")
    except pd.errors.EmptyDataError:
        print("The CSV file is empty!")
    except Exception as e:
        print(f"Error loading CSV file: {e}")

- Path to the raw post data file retrieved from Stack Overflow.
- If you don't have the data file yet, you can download it from the following: [Raw-Post-Data](https://drive.google.com/file/d/1xuHzJZrDAYPkjdxMhsSs5szoOy0cBJfJ/view?usp=drive_link) 

In [69]:
raw_post_data_path = './raw_data/posts_1M.csv'

- Load data into a DataFrame.

In [70]:
df_post = load_csv_to_dataframe(raw_post_data_path)

Data has been successfully loaded!
Number row: 1000000, Number column: 12


In [71]:
print(df_post.head(5))

      Id  PostTypeId  ParentId  \
0  38779           1       NaN   
1  38781           2   23930.0   
2  38784           1       NaN   
3  38785           2   38784.0   
4  38787           2   38769.0   

                                              Title  \
0  What SPN do I need to set for a net.tcp service?   
1                                               NaN   
2      Visual Studio equivalent to Delphi bookmarks   
3                                               NaN   
4                                               NaN   

                                                Body  \
0  <p>I have a wcf application hosted in a window...   
1  <p>Agda 2: Functional, dependently typed.</p>\...   
2  <p>I use <strong>Delphi</strong> for many year...   
3  <p><kbd>Ctrl</kbd> <kbd>K</kbd> + <kbd>Ctrl</k...   
4  <p>MPP does have its own object model that can...   

                                     Tags  Score  ViewCount  AnswerCount  \
0                      |wcf|security|spn|      6   

## 4. Extract question posts from the data.

- From the data, we can observe that only question posts contain the 'tags' attribute. Therefore, we only need to extract question posts to prepare the data for building the post classification model.

In [72]:
df_post = df_post[df_post["PostTypeId"] == 1]

In [73]:
print("Number of questions: ", len(df_post))

Number of questions:  246767


## 5. Tag analysis

- The 'tags' column is a string where individual tags are separated by the '|' character. Therefore, it should be standardized into a list format for further processing.

In [74]:
def extract_tags(tag_str):
    if pd.isna(tag_str):
        return []
    return [t for t in tag_str.strip("|").split("|") if t]

In [75]:
df_post["Tags"] = df_post["Tags"].apply(extract_tags)

In [76]:
print(df_post.head(3))

      Id  PostTypeId  ParentId  \
0  38779           1       NaN   
2  38784           1       NaN   
5  38789           1       NaN   

                                              Title  \
0  What SPN do I need to set for a net.tcp service?   
2      Visual Studio equivalent to Delphi bookmarks   
5              Web Service Namespace Dynamic Naming   

                                                Body  \
0  <p>I have a wcf application hosted in a window...   
2  <p>I use <strong>Delphi</strong> for many year...   
5  <p>I have a <code>web-service</code> that I wi...   

                                       Tags  Score  ViewCount  AnswerCount  \
0                      [wcf, security, spn]      6     6282.0          2.0   
2  [visual-studio, delphi, brief-bookmarks]      7     3474.0          8.0   
5               [c#, asp.net, web-services]      4     4406.0          1.0   

   CommentCount             CreationDate  OwnerUserId  
0             0  2008-09-02T03:41:06.880        

- Analyze tag data in the post dataset.
- Count the frequency of each tag

In [77]:
from collections import Counter
from itertools import chain

In [78]:
tag_counter = Counter(chain.from_iterable(df_post["Tags"]))
tag_counts = pd.Series(tag_counter).sort_values(ascending=False)

In [79]:
print("Number of subject: ", len(tag_counts))
print("Top 10 tags:", tag_counts.head(10).to_dict())
print("Bottom 10 tags:", tag_counts.tail(10).to_dict())

Number of subject:  16226
Top 10 tags: {'c#': 34409, '.net': 20357, 'java': 17588, 'asp.net': 15644, 'javascript': 12816, 'php': 11955, 'c++': 11926, 'python': 8972, 'sql': 8391, 'jquery': 7943}
Bottom 10 tags: {'approximate-nn-searching': 1, 'jconnect': 1, 'precision-recall': 1, 'fpc': 1, 'mousepress': 1, 'antplus': 1, 'nike': 1, 'tool-rec': 1, 'web-safe-fonts': 1, 'squarespace': 1}


- Count the number of tags that appear less than 100 times.

In [80]:
num_tags_less_100 = (tag_counts < 100).sum()
print("Number of tags with frequency < 100:", num_tags_less_100)

Number of tags with frequency < 100: 15344


- Remove all tags with a frequency less than 100 from df_post["Tags"].
- If any row has an empty or blank Tags column after filtering (i.e., an empty list or missing value), remove that row from the DataFrame.

In [81]:
def filter_tags(df_post, min_count=100):
    tag_counter = Counter(chain.from_iterable(df_post["Tags"]))
    valid_tags = {tag for tag, count in tag_counter.items() if count >= min_count}
    df_post["Tags"] = df_post["Tags"].apply(
        lambda tags: [t for t in tags if t in valid_tags]
    )
    df_post = df_post[df_post["Tags"].apply(lambda x: len(x) > 0 if isinstance(x, list) else False)]
    df_post = df_post.reset_index(drop=True)
    return df_post

In [82]:
df_post = filter_tags(df_post, min_count=1500)

In [83]:
tag_counter = Counter(chain.from_iterable(df_post["Tags"]))
tag_counts = pd.Series(tag_counter).sort_values(ascending=False)

In [84]:
print("Number row: ", len(df_post))
print("Number of subject: ", len(tag_counts))
print("Top 10 tags:", tag_counts.head(10).to_dict())
print("Bottom 10 tags:", tag_counts.tail(10).to_dict())

Number row:  204591
Number of subject:  67
Top 10 tags: {'c#': 34409, '.net': 20357, 'java': 17588, 'asp.net': 15644, 'javascript': 12816, 'php': 11955, 'c++': 11926, 'python': 8972, 'sql': 8391, 'jquery': 7943}
Bottom 10 tags: {'eclipse': 1816, 'macos': 1810, 'silverlight': 1794, 'actionscript-3': 1708, 't-sql': 1675, 'language-agnostic': 1627, 'excel': 1606, 'internet-explorer': 1521, 'nhibernate': 1510, 'exception': 1508}


In [85]:
print(df_post.head(5))

      Id  PostTypeId  ParentId  \
0  38779           1       NaN   
1  38784           1       NaN   
2  38789           1       NaN   
3  38801           1       NaN   
4  38820           1       NaN   

                                              Title  \
0  What SPN do I need to set for a net.tcp service?   
1      Visual Studio equivalent to Delphi bookmarks   
2              Web Service Namespace Dynamic Naming   
3      SQL - How to store and navigate hierarchies?   
4                     Which class design is better?   

                                                Body  \
0  <p>I have a wcf application hosted in a window...   
1  <p>I use <strong>Delphi</strong> for many year...   
2  <p>I have a <code>web-service</code> that I wi...   
3  <p>What are the ways that you use to model and...   
4  <p>Which class design is better and why?</p>\n...   

                          Tags  Score  ViewCount  AnswerCount  CommentCount  \
0              [wcf, security]      6     6282.0

## 6. Standardize the Body field

- The data in the 'Body' attribute is in HTML format and needs to be converted to plain text.

In [86]:
from ConverHtmlToText import HTMLTextExtractor

In [87]:
def html_to_text(html_string: str) -> str:
    parser = HTMLTextExtractor()
    parser.feed(html_string or "")
    return parser.get_text()

In [88]:
df_post["Body"] = df_post["Body"].apply(html_to_text)

In [89]:
print(df_post.head(5))

      Id  PostTypeId  ParentId  \
0  38779           1       NaN   
1  38784           1       NaN   
2  38789           1       NaN   
3  38801           1       NaN   
4  38820           1       NaN   

                                              Title  \
0  What SPN do I need to set for a net.tcp service?   
1      Visual Studio equivalent to Delphi bookmarks   
2              Web Service Namespace Dynamic Naming   
3      SQL - How to store and navigate hierarchies?   
4                     Which class design is better?   

                                                Body  \
0  I have a wcf application hosted in a windows s...   
1  I use Delphi for many years, and although I ha...   
2  I have a\nweb-service\n that I will be deployi...   
3  What are the ways that you use to model and re...   
4  Which class design is better and why?\n\npubli...   

                          Tags  Score  ViewCount  AnswerCount  CommentCount  \
0              [wcf, security]      6     6282.0

## 7. Remove unnecessary attributes

- The task we are solving is post classification based on content, so we only retain the 'Body' and 'Tags' attributes.

In [90]:
drop_list = ["Id", "PostTypeId", "ParentId", "Title", "Score", "ViewCount", "AnswerCount", "CommentCount", "CreationDate", "OwnerUserId"]

In [91]:
df_post = df_post.drop(columns=drop_list, errors="ignore")

In [92]:
print(df_post.head(5))

                                                Body  \
0  I have a wcf application hosted in a windows s...   
1  I use Delphi for many years, and although I ha...   
2  I have a\nweb-service\n that I will be deployi...   
3  What are the ways that you use to model and re...   
4  Which class design is better and why?\n\npubli...   

                          Tags  
0              [wcf, security]  
1      [visual-studio, delphi]  
2  [c#, asp.net, web-services]  
3    [sql, sql-server, oracle]  
4                        [oop]  


## 8. Export the results to a CSV file.

In [93]:
import os

In [94]:
def export_to_csv(df, file_path, index=False, encoding='utf-8'):
    try:
        folder = os.path.dirname(file_path)
        if folder and not os.path.exists(folder):
            os.makedirs(folder)
            print(f"Folder created successfully: {folder}")
        
        df.to_csv(file_path, index=index, encoding=encoding)
        print(f"CSV file exported successfully.: {file_path}")
        
    except Exception as e:
        print(f"Error exporting CSV file.: {e}")

In [95]:
output_path = "./dataset/post.csv"

In [96]:
export_to_csv(df_post, output_path)

CSV file exported successfully.: ./dataset/post.csv
